# Set-up
## Import all packages

In [14]:
import pandas as pd
import numpy as np
import glob
import os
from flask import Flask
import sys

import geopandas
import geopy
from geopy.geocoders import Nominatim
import folium
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from dash import Dash
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import flask

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from datetime import timedelta
import glob
import os
from tqdm import tqdm as tqdm
import time
import seaborn as sns

%matplotlib inline
import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.formula.api import ols
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings('ignore')

from numpy import cumsum
from datetime import datetime as dt

## Create functions

In [2]:
def inTimePeriod(file_path, start_year, start_month, end_year, end_month):
    file_path_parts = file_path.split("\\")
    file_name = file_path_parts[len(file_path_parts) - 1]
    year = int(file_name[:4])
    month = int(file_name[5:7])
    start_bool = (year >= start_year) or (year == start_year and month >= start_month)
    end_bool = (year <= end_year) or (year == end_year and month <= end_month)
    if (start_bool and end_bool):
        return True
    return False

def monthYearToYear(monthYear):
    return int(monthYear[:4])

def monthYearToMonth(monthYear):
    return int(monthYear[5:7])

def MonthYearRegionDF(dataframe, month, year, region):
    return dataframe[(dataframe['Month'] == month) & (dataframe['Year'] == year) & 
                           (dataframe['Falls within'] == region)]

def ARIMA_DATA(df, MSOA, category):
    ###Gets the data of a specific MSOA and category to use in the ARIMA_OPTIMAL function
    ###
    
    df = df[(df['MSOA'] == int(MSOA)) & (df['Crime type'] == category)]
    df = df[['Date', 'count']]
    df = df.set_index('Date')
    
    return df

def ARIMA_STATIONARY(df):
    ###Returns a stationary dataframe, created by ARIMA_DATA
    ###
    if adfuller(df['count'])[1] > 0.05:
        df = df.diff().dropna()
    
    return df

def ARIMA_OPTIMAL(stationary_data, MSOA, category):
    ### Looks for the best ARMA(p,q) + constant model according to MSOA and crime type
    ###
    
    order_aic_bic = list()

    # Loop over AR order
    for p in range(1,4):
        # Loop over MA order
        for q in range(1,4):
            #for d in range(3):
            try:
            # Fit model
                model = SARIMAX(stationary_data, order=(p,0,q), trend='c')
                results = model.fit()
                # Add order and scores to list
                order_aic_bic.append((p, q, results.aic))
            except:
                continue
            
    order_df = pd.DataFrame(order_aic_bic, columns=['p','q','aic'])
    optimum = order_df[order_df['aic'] == order_df['aic'].min()]
    optimum.reset_index(inplace=True)
    return optimum['p'][0], optimum['q'][0], optimum['aic'][0]

def ARIMA_PREDICT(df, MSOA, category):
    ###Forecasts via ARIMA approach
    ###
    
    arima_data = ARIMA_DATA(df, MSOA, category)
    stationary_data = ARIMA_STATIONARY(arima_data)
    
    p,q = ARIMA_OPTIMAL(stationary_data, MSOA, category)[0:2]
    
    model = SARIMAX(stationary_data, order=(p,0,q), trend='c')
    results = model.fit()
    forecast = results.get_prediction(start=-25)
    mean_forecast = cumsum(forecast.predicted_mean) + stationary_data.iloc[-1,0]
    confidence_intervals = cumsum(forecast.conf_int())
    return arima_data, mean_forecast.to_frame(), confidence_intervals

def ARIMA_SUMMARY(df, MSOA, category):
    
    stationary_data = ARIMA_STATIONARY(ARIMA_DATA(df, MSOA, category))
    
    p,q = ARIMA_OPTIMAL(stationary_data, MSOA, category)[0:2]
    
    model = SARIMAX(stationary_data, order=(p,0,q), trend='c')
    results = model.fit()
    
    return results.summary()

def model_predict(df, msoa, category):
    data, mean_forecasts, confidence_intervals = ARIMA_PREDICT(df, msoa, category)
    
    lower_limits = confidence_intervals['lower count']
    upper_limits = confidence_intervals['upper count']
    
    return data, mean_forecasts, confidence_intervals, lower_limits, upper_limits

## Import data

In [3]:
path = r'C:\Users\20201222\Courses\Y2\Data challenge 2\all_months'

In [4]:
all_streets = glob.glob(path + "/2012*street.csv")
df_streets = pd.concat(map(pd.read_csv, all_streets))
df_streets

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2012-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.516919,51.423683,On or near A4175,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
1,NaN,2012-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.510162,51.410998,On or near Monmouth Road,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
2,NaN,2012-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.511927,51.409435,On or near Harlech Close,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
3,b1a34824199f9d587ef05668511759f3cb9e69a35e9842...,2012-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.494870,51.422276,On or near Conference/Exhibition Centre,E01014399,Bath and North East Somerset 001A,Other theft,Offender given community sentence,NaN
4,67ea8d3cff9fcf5bbea27e1bdab08608ea141883c23ad6...,2012-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.510162,51.410998,On or near Monmouth Road,E01014399,Bath and North East Somerset 001A,Other theft,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4193,1618327bab236c64b5a1300088c965a0e60ab19ac45066...,2012-12,Wiltshire Police,Wiltshire Police,-1.756865,50.993382,On or near Long Close West,E01031995,Wiltshire 062E,Criminal damage and arson,Investigation complete; no suspect identified,NaN
4194,2b1d109370ad5274229052b704409b157b8a421a743ede...,2012-12,Wiltshire Police,Wiltshire Police,-1.759025,50.994654,On or near Petrol Station,E01031995,Wiltshire 062E,Other theft,Investigation complete; no suspect identified,NaN
4195,b5a6704a605b9a8d0f12fbd9d2eaaa92e0de4febefab6c...,2012-12,Wiltshire Police,Wiltshire Police,-1.751553,50.992849,On or near The Borough,E01031995,Wiltshire 062E,Shoplifting,Investigation complete; no suspect identified,NaN
4196,f59fcf9bc9efd837919e91d46adba04bc874db94b3d961...,2012-12,Wiltshire Police,Wiltshire Police,-1.742939,50.996850,On or near Sports/Recreation Area,E01031995,Wiltshire 062E,Vehicle crime,Investigation complete; no suspect identified,NaN


In [ ]:
data_path = r'C:\Users\20201222\Courses\Y2\Data challenge 2'
Train = pd.read_csv(str(data_path) + r"\\covid_train.csv")
Train.drop("Unnamed: 0", axis=1, inplace=True)

#TestCovid = pd.read_csv(data_path + r"\\covid_test.csv")
#TestCovid.drop("Unnamed: 0", axis=1, inplace=True)

#TestNoCovid = pd.read_csv(data_path + r"\\no_covid_test.csv")
#TestNoCovid.drop("Unnamed: 0", axis=1, inplace=True)

#TrainWithNoCovid = pd.read_csv(data_path + r"\\no_covid_train.csv")
#TrainWithNoCovid.drop("Unnamed: 0", axis=1, inplace=True)
Train

In [6]:
best_models_covid = pd.read_csv(str(data_path) + r"\\best_models.csv")
best_models_covid

,Unnamed: 0,MSOA,Crime type,p,q,aic,MASE,Unnamed: 0.1
0,0,"Abberley, Holt Heath & Hallow",Anti-social behaviour,2,2,489.384443,2.331706,0.0
1,1,"Abberley, Holt Heath & Hallow",Bicycle theft,1,1,31.219505,0.589988,1.0
2,2,"Abberley, Holt Heath & Hallow",Burglary,3,3,426.688176,0.826900,2.0
3,3,"Abberley, Holt Heath & Hallow",Criminal damage and arson,2,2,341.449159,0.847437,3.0
4,4,"Abberley, Holt Heath & Hallow",Drugs,3,4,121.634511,0.828079,4.0
...,...,...,...,...,...,...,...,...
99857,99857,Ystradgynlais & Tawe Uchaf,Robbery,1,1,127.626177,0.651113,12441.0
99858,99858,Ystradgynlais & Tawe Uchaf,Shoplifting,1,1,269.478958,1.588960,12442.0
99859,99859,Ystradgynlais & Tawe Uchaf,Theft from the person,2,2,-28.011987,1.756758,12443.0
99860,99860,Ystradgynlais & Tawe Uchaf,Vehicle crime,2,2,349.912842,1.322706,12444.0


## Preperation
### Drop unecessary columns

In [7]:
df_streets = df_streets.drop(columns=['Context', 'Location', 'Reported by', 'Last outcome category', 'LSOA code'])
df_streets

,Crime ID,Month,Falls within,Longitude,Latitude,LSOA name,Crime type
0,NaN,2012-01,Avon and Somerset Constabulary,-2.516919,51.423683,Bath and North East Somerset 001A,Anti-social behaviour
1,NaN,2012-01,Avon and Somerset Constabulary,-2.510162,51.410998,Bath and North East Somerset 001A,Anti-social behaviour
2,NaN,2012-01,Avon and Somerset Constabulary,-2.511927,51.409435,Bath and North East Somerset 001A,Anti-social behaviour
3,b1a34824199f9d587ef05668511759f3cb9e69a35e9842...,2012-01,Avon and Somerset Constabulary,-2.494870,51.422276,Bath and North East Somerset 001A,Other theft
4,67ea8d3cff9fcf5bbea27e1bdab08608ea141883c23ad6...,2012-01,Avon and Somerset Constabulary,-2.510162,51.410998,Bath and North East Somerset 001A,Other theft
...,...,...,...,...,...,...,...
4193,1618327bab236c64b5a1300088c965a0e60ab19ac45066...,2012-12,Wiltshire Police,-1.756865,50.993382,Wiltshire 062E,Criminal damage and arson
4194,2b1d109370ad5274229052b704409b157b8a421a743ede...,2012-12,Wiltshire Police,-1.759025,50.994654,Wiltshire 062E,Other theft
4195,b5a6704a605b9a8d0f12fbd9d2eaaa92e0de4febefab6c...,2012-12,Wiltshire Police,-1.751553,50.992849,Wiltshire 062E,Shoplifting
4196,f59fcf9bc9efd837919e91d46adba04bc874db94b3d961...,2012-12,Wiltshire Police,-1.742939,50.996850,Wiltshire 062E,Vehicle crime


### Create Month and Year column

In [8]:
df_streets['Year'] = df_streets['Month'].apply(monthYearToYear)
df_streets['Month'] = df_streets['Month'].apply(monthYearToMonth)
df_streets

,Crime ID,Month,Falls within,Longitude,Latitude,LSOA name,Crime type,Year
0,NaN,1,Avon and Somerset Constabulary,-2.516919,51.423683,Bath and North East Somerset 001A,Anti-social behaviour,2012
1,NaN,1,Avon and Somerset Constabulary,-2.510162,51.410998,Bath and North East Somerset 001A,Anti-social behaviour,2012
2,NaN,1,Avon and Somerset Constabulary,-2.511927,51.409435,Bath and North East Somerset 001A,Anti-social behaviour,2012
3,b1a34824199f9d587ef05668511759f3cb9e69a35e9842...,1,Avon and Somerset Constabulary,-2.494870,51.422276,Bath and North East Somerset 001A,Other theft,2012
4,67ea8d3cff9fcf5bbea27e1bdab08608ea141883c23ad6...,1,Avon and Somerset Constabulary,-2.510162,51.410998,Bath and North East Somerset 001A,Other theft,2012
...,...,...,...,...,...,...,...,...
4193,1618327bab236c64b5a1300088c965a0e60ab19ac45066...,12,Wiltshire Police,-1.756865,50.993382,Wiltshire 062E,Criminal damage and arson,2012
4194,2b1d109370ad5274229052b704409b157b8a421a743ede...,12,Wiltshire Police,-1.759025,50.994654,Wiltshire 062E,Other theft,2012
4195,b5a6704a605b9a8d0f12fbd9d2eaaa92e0de4febefab6c...,12,Wiltshire Police,-1.751553,50.992849,Wiltshire 062E,Shoplifting,2012
4196,f59fcf9bc9efd837919e91d46adba04bc874db94b3d961...,12,Wiltshire Police,-1.742939,50.996850,Wiltshire 062E,Vehicle crime,2012


# Dashboard
## Elements

In [37]:
# Getting lists
police_list = df_streets['Falls within'].sort_values(ascending=True).unique().tolist() #police force list
crime_list = df_streets['Crime type'].sort_values(ascending=True).unique().tolist() #crime type list
msoa_list = best_models_covid['MSOA'].sort_values(ascending=True).unique().tolist() #MSOA list
year_list = df_streets['Year'].sort_values(ascending=True).unique().tolist() #year list
month_list = df_streets['Month'].sort_values(ascending=True).unique().tolist() #month list

In [10]:
#Dataframe 
##Bar
df_selected = df_streets.loc[df_streets['Falls within'] == 'Avon and Somerset Constabulary']
df_grouped_bar = df_selected.groupby(['Falls within', 'Crime type']).agg({'Month' : 'count'})
df_grouped_bar_new = df_grouped_bar.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
df_grouped_bar_new['Number of crimes'] = df_grouped_bar['Month']
df_grouped_bar_new = df_grouped_bar_new.squeeze()
df_grouped_bar_perc = df_grouped_bar_new.rename(columns = {'Month' : 'Percentage crimes per police force'}).reset_index()
df_grouped_bar_perc = df_grouped_bar_perc.round(2)

#Line chart
data, mean_forecasts, confidence_intervals, lower_limits, upper_limits = model_predict(Train, '001', 'Anti-social behaviour')
    
test = mean_forecasts.reset_index()
test['Date'] = test['index'].dt.strftime('%Y-%m')
test.drop(columns=['index'], inplace=True, axis=1)
test.set_index('Date', inplace=True)
test = test.rename(columns={0:'predicted_mean'})

plot_data = pd.concat([data, test], axis=1)

In [11]:
#Bar chart
bar_chart = px.bar(data_frame=df_grouped_bar_perc,
                    x = 'Falls within',
                    y = 'Percentage crimes per police force',
                    color = 'Crime type',
                    barmode='group',
                    orientation = 'v',
                    text_auto=True,
                    hover_name="Falls within", 
                    hover_data={
                        'Falls within': False,
                        'Number of crimes' : True
                    },
                   color_discrete_sequence=px.colors.qualitative.Safe,
                   title = 'Title',
                   )

#Heatmap
hm = folium.Map([53.5500,-2.4333], zoom_start=6) 
hm.save('Heatmap.html')

#Line chart
line_chart = px.line(data_frame = plot_data, 
                     x=plot_data.index, 
                     y=plot_data.columns[0:2])

#Graph
bar_graph = dcc.Graph(
        id='bar_graph',
        figure=bar_chart,
    )

heat_graph = html.Iframe(id = 'heat_graph',
                        srcDoc = open('Heatmap.html', 'r',).read(),
                        width = '1200',
                        height = '600')

line_graph = dcc.Graph(
        id='line_graph',
        figure=line_chart,
    )

In [15]:
#Widgets
police_dropdown = dcc.Dropdown(
    id = 'police_dropdown',
    options = police_list, 
    multi = True,
)
police_dropdown2 = dcc.Dropdown(
    id = 'police_dropdown2',
    options = police_list, 
)

msoa_dropdown = dcc.Dropdown(
    id = 'msoa_dropdown',
    options = msoa_list, 
    value = msoa_list[0],)

crime_checklist = dcc.Checklist(
    id = 'crime_checklist',
    options = crime_list, 
    value = crime_list,
    labelStyle = dict(display='block'))
crime_radio = dcc.RadioItems(
    id = 'crime_radio',
    options = crime_list, 
    value = crime_list[0],
    labelStyle = dict(display='block'))
crime_radio2 = dcc.RadioItems(
    id = 'crime_radio2',
    options = crime_list, 
    value = crime_list[0],
    labelStyle = dict(display='block'))

year_dropdown = dcc.Dropdown(
    id = 'year_dropdown',
    options = year_list, 
    value = year_list[0],)
month_dropdown = dcc.Dropdown(
    id = 'month_dropdown',
    options = month_list, 
    value = month_list[0],)

deselect_button = html.Button("Deselect all", id="deselect_button")

value_type_radio = dcc.RadioItems(
    id  = 'value_type_radio',
    options = ['Percentages', 'Absolute values'], 
    value = 'Percentages',
)

values = html.Div(
    id = 'values',
    children = None,)

p_tool = html.Div(
    [html.Div(id='p_value', 
              children=['P: {}'.format(None)]),
    dbc.Tooltip("The number of lag observations in the model; also known as the lag orde.",
                target="p_value"),
    ])
q_tool = html.Div(
    [html.Div(id='q_value', 
              children=['Q: {}'.format(None)]),
    dbc.Tooltip("The size of the moving average window; also known as the order of the moving average.",
                target="q_value"),
    ])
aic_tool = html.Div(
    [html.Div(id='aic_value', 
              children=['AIC: {}'.format(None)]),
    dbc.Tooltip("The Akaike information criterion (AIC) is an estimator of out-of-sample prediction error and \
                thereby relative quality of statistical models for a given set of data.",
                target="aic_value"),
    ])
mase_tool = html.Div(
    [html.Div(id='mase_value', 
              children=['MASE: {}'.format(None)]),
    dbc.Tooltip("Mean Absolute Scaled Error (MASE) is a scale-free error metric that gives each error as a ratio \
                compared to a baseline’s average error.",
                target="mase_value"),
    ])

submit_button = html.Button("Submit changes to map", id="submit_button")

In [16]:
#Create several widgets and graphs layout
## Tab 1
widgets1 = [html.H2('Widgets', style={'font-family' : 'Verdana'}),
           html.H3('Location:', style={'font-family' : 'Verdana'}),
               dcc.Markdown('Police force:', style={'font-family' : 'Verdana'}), police_dropdown,
           html.Br(),
           html.H3('Crime type:', style={'font-family' : 'Verdana'}), \
               deselect_button, crime_checklist,
           html.Br(),
           html.H3('Unit crimes:', style={'font-family' : 'Verdana'}), value_type_radio
           ]
graphs1 = [html.H2('Graph', style={'font-family' : 'Verdana'}),
          bar_graph]

## Tab 2
widgets2 = [html.H2('Widgets', style={'font-family' : 'Verdana'}),
           html.H3('Location:', style={'font-family' : 'Verdana'}),
            dcc.Markdown('Police force:', style={'font-family' : 'Verdana'}), police_dropdown2,
           html.Br(),
           html.H3('Crime type:', style={'font-family' : 'Verdana'}), crime_radio,
           html.Br(),
           html.H3('Time:', style={'font-family' : 'Verdana'}), 
            dcc.Markdown('Month:', style={'font-family' : 'Verdana'}), month_dropdown, 
            dcc.Markdown('Year:', style={'font-family' : 'Verdana'}), year_dropdown, 
            html.Br(),
           submit_button]
graphs2 = [html.H2('Graph', style={'font-family' : 'Verdana'}),
           heat_graph ]

## Tab 3
widgets3 = [html.H2('Widgets', style={'font-family' : 'Verdana'}),
            html.H3('Location:', style={'font-family' : 'Verdana'}), 
            dcc.Markdown('Select corresponding MSOA:', style={'font-family' : 'Verdana'}), msoa_dropdown,
            html.Br(),
            html.H3('Crime type:', style={'font-family' : 'Verdana'}), crime_radio2,
            html.Br(),
            html.H3('ARIMA model statistics:', style={'font-family' : 'Verdana'}), values, 
            html.Br(), p_tool, 
            html.Br(), q_tool, 
            html.Br(), aic_tool, 
            html.Br(), mase_tool
           ]
        
graphs3 = [html.H2('Graph', style={'font-family' : 'Verdana'}),
          line_graph]

## Update charts

In [17]:
# server = flask.Flask(__name__)
# app = dash.Dash(__name__, server=server)
app = dash.Dash(__name__)

@app.callback(
    Output('bar_graph', 'figure'),
    [Input("police_dropdown", "value"), Input("crime_checklist", "value"), Input("value_type_radio", "value")]
)
def update_bar(police_force, crime_type, value_type):
    ##update dataframes
    df_selected = df_streets.loc[df_streets['Falls within'].isin(police_force) & \
    df_streets['Crime type'].isin(crime_type)]
    df_grouped = df_selected.groupby(['Falls within', 'Crime type']).agg({'Month' : 'count'})
    df_grouped_new = df_grouped.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
    df_grouped_new['Number of crimes'] = df_grouped['Month']
    df_grouped_new = df_grouped_new.squeeze()
    df_grouped_perc = df_grouped_new.rename(columns = {'Month' : 'Percentage crimes per police force'}).reset_index()
    df_grouped_perc = df_grouped_perc.round(2)
    
    if value_type == 'Absolute values':
        y_value = 'Number of crimes'
    elif value_type == 'Percentages':
        y_value = 'Percentage crimes per police force'
    
    ##update graphs
    chart = px.bar(data_frame=df_grouped_perc,
                    x = 'Falls within',
                    y = y_value,
                    color = 'Crime type',
                    barmode='group',
                    orientation = 'v',
                    text_auto=True,
                    hover_name="Falls within", 
                    hover_data={
                        'Falls within': False,
                        'Percentage crimes per police force' : True,
                        'Number of crimes' : True
                    },
                   color_discrete_sequence=px.colors.qualitative.Safe,
                   title = 'Title',
                   )
    return chart
    
@app.callback(
    Output('crime_checklist', "value"),
    [Input('deselect_button', "n_clicks")]
)
def deselect_crime_type(n_clicks):
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate
    else:
        return []

In [18]:
@app.callback(
    [Output('heat_graph', 'srcDoc'), Output('submit_button', "n_clicks")],
    [Input("police_dropdown2", "value"), Input("crime_radio", "value"), \
     Input('year_dropdown', 'value'), Input('month_dropdown', 'value'),
     Input('submit_button', "n_clicks")
    ]
)
def update_heat(police_force, crime_type, year, month, n_clicks):
    ##update dataframes
    df_grouped_heat = MonthYearRegionDF(df_streets, month, year, police_force)
    df_grouped_heat2 = df_grouped_heat[df_grouped_heat['Crime type'] == crime_type].dropna()
    heat_data = [[row['Latitude'],row['Longitude']] for index, row in df_grouped_heat2.iterrows()]
    
    hm = folium.Map([53.5500,-2.4333], zoom_start=6) 
    HeatMap(heat_data).add_to(hm)
    hm.save('Heatmap.html')
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate
    else:
        return open('Heatmap.html', 'r',).read(), None

In [32]:
@app.callback(
    Output('line_graph', 'figure'),
    [Input("msoa_dropdown", "value"), Input("crime_radio2", "value")]
)
def update_line(MSOA, crime):
    #recalculating the data
    data, mean_forecasts, confidence_intervals, lower_limits, upper_limits = model_predict(Train, MSOA, crime)
    
    test = mean_forecasts.reset_index()
    test['Date'] = test['index'].dt.strftime('%Y-%m')
    test.drop(columns=['index'], inplace=True, axis=1)
    test.set_index('Date', inplace=True)
    test = test.rename(columns={0:'predicted_mean'})
    plot_data = pd.concat([data, test], axis=1)
    
    # updating chart
    line_chart = px.line(data_frame = plot_data, 
                     x=plot_data.index, 
                     y=plot_data.columns[0:2])
    
    return line_chart


@app.callback(
    [Output('p_value', 'children'), Output('q_value', 'children'), Output('aic_value', 'children'), Output('mase_value', 'children')],
    [Input("msoa_dropdown", "value"), Input("crime_radio2", "value")]
)
def update_values(MSOA, crime):
    p = best_models_covid[(best_models_covid['MSOA']==MSOA) & (best_models_covid['Crime type']==crime)]['p']
    p_str = f'P: {p}'
    
    q = best_models_covid[
        best_models_covid['MSOA']==MSOA][
        best_models_covid['Crime type']==crime][
        'q']
    q_str = f'Q: {q}'

    AIC = best_models_covid[
        best_models_covid['MSOA']==MSOA][
        best_models_covid['Crime type']==crime][
        'aic']
    aic_str = f'AIC: {AIC}'

    MASE = best_models_covid[
        best_models_covid['MSOA']==MSOA][
        best_models_covid['Crime type']==crime][
        'MASE']
    mase_str = f'MASE: {MASE}'
    
    return p_str, q_str, aic_str, mase_str

## Layout dashboard

In [33]:
#Create column layouts
widget_layout1 = html.Div(children = widgets1, style = {'width' : '20%', 'display' : 'inline-block'})
graph_layout1 = html.Div(children = graphs1, style = {'width' : '78%', 'display' : 'inline-block', 'float' : 'right'})

widget_layout2 = html.Div(children = widgets2, style = {'width' : '20%', 'display' : 'inline-block'})
graph_layout2 = html.Div(children = graphs2, style = {'width' : '78%', 'display' : 'inline-block', 'float' : 'right'})

widget_layout3 = html.Div(children = widgets3, style = {'width' : '20%', 'display' : 'inline-block'})
graph_layout3 = html.Div(children = graphs3, style = {'width' : '78%', 'display' : 'inline-block', 'float' : 'right'})

In [34]:
#Create full layout
full_layout = [html.H1("Title", style={'textAlign': 'center', 'font-family' : 'Verdana'}),
               dcc.Tabs(id="tabs", value='tab_1', children=[
                dcc.Tab(label = 'Analysis: Comparison', id='tab_1', value = 'tab_1', children = [widget_layout1, graph_layout1]), 
                dcc.Tab(label = 'Analysis: Hotspots', id='tab_2', children = [widget_layout2, graph_layout2]),
                dcc.Tab(label = 'Model: ARIMA forecasting', id='tab_3', children = [widget_layout3, graph_layout3])
               ]),
            html.Div(id='tabs_content')
            ]
#Create layout
app.layout = html.Div(children = full_layout)

## Run application

In [35]:
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2022 12:54:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 12:54:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 12:54:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 12:54:19] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2022 12:54:19] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2022 12:54:19] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2022 12:54:19] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2022 12:54:19] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Apr/2022 12:54:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 12:54:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 12:54:36] "GET /_dash-depende

In [ ]:
# Extra cell in case previous cell gets deleted
# if __name__ == "__main__":
#     app.run_server(debug=False)